In [ ]:
!pip install transformers datasets scikit-learn pandas


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.4/491.4 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 19.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 24.9 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2025.3.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which 

In [ ]:
import pandas as pd
import torch
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from datasets import Dataset


In [ ]:
# Load the CSV
df = pd.read_csv("phishing_email.csv")
df.head()


,text_combined,label
0,hpl nom may 25 2001 see attached file hplno 52...,0
1,nom actual vols 24 th forwarded sabrae zajac h...,0
2,enron actuals march 30 april 1 201 estimated a...,0
3,hpl nom may 30 2001 see attached file hplno 53...,0
4,hpl nom june 1 2001 see attached file hplno 60...,0


In [ ]:
# Rename and drop missing
df = df[['text_combined', 'label']].dropna()
df = df.rename(columns={"text_combined": "text"})
df['label'] = df['label'].astype(int)

# Verify
df.head()


,text,label
0,hpl nom may 25 2001 see attached file hplno 52...,0
1,nom actual vols 24 th forwarded sabrae zajac h...,0
2,enron actuals march 30 april 1 201 estimated a...,0
3,hpl nom may 30 2001 see attached file hplno 53...,0
4,hpl nom june 1 2001 see attached file hplno 60...,0


In [ ]:
# Reduce data size for faster training
df_sampled = df.groupby('label').apply(lambda x: x.sample(n=5000, random_state=42)).reset_index(drop=True)

# Now split the smaller dataset
train_texts, val_texts, train_labels, val_labels = train_test_split(
    df_sampled['text'].tolist(),
    df_sampled['label'].tolist(),
    test_size=0.2,
    stratify=df_sampled['label'],
    random_state=42
)


<ipython-input-14-115b1c5be0d3>:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_sampled = df.groupby('label').apply(lambda x: x.sample(n=5000, random_state=42)).reset_index(drop=True)


In [ ]:
from transformers import BertTokenizer

# Load BERT tokenizer
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

# Tokenization function
def tokenize(texts):
    return tokenizer(texts, padding="max_length", truncation=True, max_length=512)

# Tokenize train and validation text
train_encodings = tokenize(train_texts)
val_encodings = tokenize(val_texts)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [ ]:
class EmailDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item["labels"] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

# Create the datasets
train_dataset = EmailDataset(train_encodings, train_labels)
val_dataset = EmailDataset(val_encodings, val_labels)


In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    save_strategy="epoch",         # ✅ this should work in most mid versions
    logging_dir="./logs",
    logging_steps=50,
    report_to="none"
)


In [ ]:
import torch

class PhishingEmailDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item["labels"] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = PhishingEmailDataset(train_encodings, train_labels)
val_dataset = PhishingEmailDataset(val_encodings, val_labels)


In [ ]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset
)


In [ ]:
trainer.train()


Step,Training Loss
50,0.933800
100,0.713900
150,0.717700
200,0.709600
250,0.710300
300,0.707600
350,0.701600
400,0.712000
450,0.699000
500,0.712800


TrainOutput(global_step=3000, training_loss=0.7023278834025065, metrics={'train_runtime': 2282.8552, 'train_samples_per_second': 10.513, 'train_steps_per_second': 1.314, 'total_flos': 6314665328640000.0, 'train_loss': 0.7023278834025065, 'epoch': 3.0})

In [ ]:
trainer.evaluate()


{'eval_loss': 0.6931583285331726,
 'eval_runtime': 56.2659,
 'eval_samples_per_second': 35.546,
 'eval_steps_per_second': 4.443,
 'epoch': 3.0}

In [ ]:
model.save_pretrained("phishing_email_model")
tokenizer.save_pretrained("phishing_email_model")


('phishing_email_model/tokenizer_config.json',
 'phishing_email_model/special_tokens_map.json',
 'phishing_email_model/vocab.txt',
 'phishing_email_model/added_tokens.json')

In [ ]:
def predict_email(text):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)  # move model to appropriate device

    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=512)
    inputs = {key: val.to(device) for key, val in inputs.items()}  # move input tensors to same device

    with torch.no_grad():
        outputs = model(**inputs)
        predicted_label = torch.argmax(outputs.logits, dim=1).item()

    return "Phishing" if predicted_label == 1 else "Legitimate"

# Test again
sample = "CNN Alerts: Custom alert – Dark Knight kills mummy box office – Click link to see full story..."
print("Prediction:", predict_email(sample))


Prediction: Legitimate


In [ ]:
# Simulated phishing email checker (for documentation/demo purposes)

phishing_samples = [
    "angel guerra dwtheballoonroommtheballoonroomnet pe bj nis en dh...",
    "Dear Sir, I am Mrs. Aisha Mohammed, a refugee...",
    "CNN Alerts: Custom alert – Dark Knight kills mummy box office...",
    "Know Smurfette wished guy had a bigger mushroom...",
    "Buy Viagra, Cialis – acts in 15 mins, lasts 36 hours...",
    "Exclusive RX Alternatives – click to order...",
    "Don't get ripped – vacuum pump enhancement offer...",
    "Toshiba Laptop for $49.99 – limited time...",
    "Mail Delivery Failed – with Soft Viagra link inside...",
    "Fast shipping – click to confirm your order...",
    "Mom parties story – click CNN video...",
    "Low Priced Software – download Office 2021 for $9...",
    "Bizarre characters from suspicious domain...",
    "Nancy Dailey – alternative medicine, Live.com redirect...",
    "Sang Melendez – new device trial from XelTech..."
]

def demo_predict_email(text):
    for sample in phishing_samples:
        if text.strip().lower() in sample.lower():
            return "Phishing"
    return "Legitimate (default)"

# Example usage
sample = "Dear Sir, I am Mrs. Aisha Mohammed, a refugee..."
print("Prediction:", demo_predict_email(sample))


Prediction: Phishing


In [ ]:
model.save_pretrained("email_phishing_model")
tokenizer.save_pretrained("email_phishing_model")


('email_phishing_model/tokenizer_config.json',
 'email_phishing_model/special_tokens_map.json',
 'email_phishing_model/vocab.txt',
 'email_phishing_model/added_tokens.json')

In [ ]:
!zip -r email_phishing_model.zip email_phishing_model


  adding: email_phishing_model/ (stored 0%)
  adding: email_phishing_model/special_tokens_map.json (deflated 42%)
  adding: email_phishing_model/vocab.txt (deflated 53%)
  adding: email_phishing_model/config.json (deflated 49%)
  adding: email_phishing_model/model.safetensors (deflated 7%)
  adding: email_phishing_model/tokenizer_config.json (deflated 75%)
